In [1]:
import os
import io 
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr
import subprocess
from IPython.display import Markdown,display,update_display

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = " claude-3-haiku-20240307"

#uncomment if you want higher models use these two.

# OPENAI_MODEL = "gpt-4o"
# CLAUDE_MODEL = "claude-3-5-sonnet-20240620"


In [4]:
system_message = "you are and assistant that implements Python code into C++ code for Intel PC with windows as operating system"
system_message += "Respond only with C++ code and only comment sparingly just to explain your work occasionally"
system_message += " The C++ responds needs to reproduce an identical output in the fastest possible time"

In [5]:
def user_prompt(python):
        user_prompt = "Rewrite this python code in C++ in the fastest time possible\n\n"
        user_prompt += "respond only in C++ and dont explain your work please."
        user_prompt += "Pay attention to the number types to ensure no overflow happens, try to use int64 number types if the numbers are large. Remember to #include all necessary packages such as iomanip.\n\n"
        user_prompt += python
        return user_prompt

In [6]:
def messages_for(python):
        return [
    
        {"role" : "system" , "content": system_message},
        {"role" : "user" , "content" : user_prompt(python)}
    
        ]

In [7]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):

    stream = openai.chat.completions.create(
        model = OPENAI_MODEL ,
        messages = messages_for(python) , 
        stream = True)
    reply = ""

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end= '' , flush = True)
        
    write_output(reply)
    

In [9]:
def optimize_claude(python):
    result = claude.messages.stream(
        model = CLAUDE_MODEL,
        max_tokens = 2000,
        messages = [{"role" : "user", "content" : user_prompt(python)}]
     )
    reply = ""
    with result as stream:
         for text in stream.text_stream:
             reply+= text
             print(text,end = '' , flush = True)
    write_output(reply)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 29.052974 seconds


In [12]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

typedef long long int64;

double calculate(int64 iterations, int64 param1, int64 param2) {
    double result = 1.0;
    for (int64 i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100000000, 4, 1) * 4;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;
    
    return 0;
}
```

In [13]:
!g++ -o optimized optimized.cpp


In [14]:
!optimized


Result: 3.141592658589
Execution Time: 4.179970400000 seconds


In [15]:
#Write your Python code here:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [16]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 85.583528 seconds


In [17]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>

int64_t lcg(int64_t& value, const int64_t a = 1664525, const int64_t c = 1013904223, const int64_t m = 1LL << 32) {
    value = (a * value + c) % m;
    return value;
}

int64_t max_subarray_sum(int64_t n, int64_t seed, int64_t min_val, int64_t max_val) {
    std::vector<int64_t> random_numbers(n);
    int64_t value = seed;

    for (int64_t i = 0; i < n; i++) {
        random_numbers[i] = (lcg(value) % (max_val - min_val + 1)) + min_val;
    }

    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (int64_t i = 0; i < n; i++) {
        int64_t current_sum = 0;
        for (int64_t j = i; j < n; j++) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

int64_t total_max_subarray_sum(int64_t n, int64_t initial_seed, int64_t min_val, int64_t max_val) {
    int64_t total

In [18]:
!g++ -o optimized optimized.cpp
!optimized


Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 6.23814 seconds


## Lets Try creating a Gradio UI for our code

In [19]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [22]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [23]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [24]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [25]:


def execute_cpp(code):
    # Save the C++ code to a file
    with open("optimized.cpp", "w") as cpp_file:
        cpp_file.write(code)

    try:
        # Compile the C++ code using g++
        compile_cmd = ["g++", "-Ofast", "-std=c++17", "-o", "optimized.exe", "optimized.cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)

        # Run the compiled executable
        run_cmd = ["optimized.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)

        # Return the output of the program
        return run_result.stdout

    except subprocess.CalledProcessError as e:
        # Return the error message if compilation or execution fails
        return f"An error occurred:\n{e.stderr}"


In [26]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [31]:
with gr.Blocks(css =css) as ui:
    gr.Markdown("##Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label = "Python code:", value = python_hard , lines = 10)
        cpp = gr.Textbox(label = "C++ code:", lines = 10)
    with gr.Row():
        model = gr.Dropdown(["GPT","Claude"],label="Select Model", value = "GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label ="Python results", elem_classes =["python"])
        cpp_out = gr.TextArea(label = "C++ results",elem_classes=["cpp"])

    convert.click(optimize , inputs=[python,model],outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
